In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT us.user_id AS user_id,
       us.lt_day AS lt_day,
       CASE
           WHEN us.lt_day <= 365 THEN '1'
           ELSE '0'
       END AS is_new,
       us.age AS age,
       CAST(us.gender_segment AS char) AS gender_segment,
       us.os_name AS os_name,
       us.cpe_type_name AS cpe_type_name,
       lc.country AS country,
       lc.city AS city,
       substr(ag.title, 4) AS age_segment, --применение функции substr
       substr(ts.title, 4) AS traffic_segment, --применение функции substr
       substr(ls.title, 4) AS lifetime_segment, --применение функции substr
       us.nps_score AS nps_score,
       CASE
           WHEN us.nps_score BETWEEN 9 AND 10 THEN 'cторонник'
           WHEN us.nps_score BETWEEN 7 AND 8 THEN 'нейтрал'
           WHEN us.nps_score BETWEEN 0 AND 6 THEN 'критики'
       END AS nps_group
FROM USER AS us
LEFT JOIN LOCATION AS lc ON us.location_id = lc.location_id
LEFT JOIN age_segment AS ag ON us.age_gr_id = ag.age_gr_id
LEFT JOIN traffic_segment AS ts ON us.tr_gr_id = ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON us.lt_gr_id = ls.lt_gr_id;
"""

Применил функцию `LEFT JOIN`, чтобы не потерять часть данных на случай, если в какой-либо правой таблице не окажется какого-либо индекса таблицы `USER`.

In [4]:
df = pd.read_sql(query, engine)

In [5]:
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонник
1,A001WF,2344,0,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонник
2,A003Q7,467,0,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонник
3,A004TB,4190,0,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонник
4,A004XT,1163,0,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонник
5,A005O0,5501,0,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,0,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,cторонник
7,A009KS,313,1,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,cторонник
8,A00AES,3238,0,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,cторонник
9,A00F70,4479,0,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,cторонник


In [6]:
#Выводим общую информацию по датафрейму df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


Видим, что почти во всех атрибутах содержатся категориальные данные, поэтому и графики на дашбордах в основном будут представлять собой количество пользователей в различных группах, например, в зависимости от возраста, длительности использования сервиса и т.д.

In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/daniil4791/viz/NPS_16483651084650/NPS_1?publish=yes

### Укажите ссылку на pdf-файл с презентацией:

https://drive.google.com/file/d/1q8VxePirauZFF5qKSf9RoB8d4hRIpZrX/view?usp=sharing